In [1]:
import sys
import json
from pathlib import Path
import sqlalchemy
import pandas as pd
from utils import * 
from tqdm.notebook import tqdm

In [2]:
def print_res(query):
    res = pd.read_sql(query, conn)
    print(res, "\n")

In [3]:
def create_trigger(trigger_name, trigger_body, func_name, func_body):
    query ="""
create or replace function {func_name}() returns trigger as ${trigger_name}$
    begin
        {func_body}
    end;
${trigger_name}$ language plpgsql;

create trigger {trigger_name}
{trigger_body};""".format(trigger_name=trigger_name, trigger_body=trigger_body.strip(),
                          func_name=func_name, func_body=func_body.strip())
    return query

In [4]:
tr1 = 'AllergyTrigger'
tr1_body = """
after insert on FoodInfo
for each row
execute procedure f1()
"""
fr1 = "f1"
fr1_body = """
        insert into AllergyProduct (select New.prdlstreportno, r.*
        from allergyrawmtrl r
        where New.rawMtrl LIKE '%' || r.rawmtrl || '%');
        return null;
"""
print(create_trigger(tr1, tr1_body, fr1, fr1_body))


create or replace function f1() returns trigger as $AllergyTrigger$
    begin
        insert into AllergyProduct (select New.prdlstreportno, r.*
        from allergyrawmtrl r
        where New.rawMtrl LIKE '%' || r.rawmtrl || '%');
        return null;
    end;
$AllergyTrigger$ language plpgsql;

create trigger AllergyTrigger
after insert on FoodInfo
for each row
execute procedure f1();


In [ ]:
tr1 = 'AllergyTrigger'
tr1_body = """
after insert on FoodInfo
for each row
execute procedure f2()
"""
fr1 = "f1"
fr1_body = """
        if  New.rawMtrl LIKE '%' || vegrawmtrl.rawmtrl || '%' then
            insert into Apply values (New.sID, 'Stanford', 'geology', null);
            insert into Apply values (New.sID, 'MIT', 'biology', null);
        end if;
        return null;
"""
print(create_trigger(tr1, tr1_body, fr1, fr1_body))